In [ ]:
import tensorflow as tf
from itertools import product

In [ ]:
def generate_combinations(NbrSources, FocalElem):
    dimensions = [range(FocalElem + 1) for _ in range(NbrSources)]
    combinations = list(product(*dimensions))
    return tf.constant(combinations)

def PCR6plusfusion(BBA):
    NbrSources = BBA.shape[1]

    CardTheta = BBA.shape[0]

    if NbrSources == 1:
        return BBA[:, 0]

    Combinations = generate_combinations(NbrSources, CardTheta)
    Combinations = tf.reverse(Combinations, axis=[1])

    mPCR6plus = tf.zeros((BBA.shape[0], 1), dtype=tf.float32)
    for c in range(Combinations.shape[0]):
        PC = Combinations[c, :]
        print("PC", PC)
        masseConj = tf.linalg.diag_part(BBA[PC, :])
        print("masseConj", masseConj)
        massConj = tf.reduce_prod(tf.linalg.diag_part(BBA[PC, :]))
        Intersections = PC[0] + 1

        for s in range(1, NbrSources):
            Intersections = tf.bitwise.bitwise_and(Intersections, PC[s] + 1)

        if Intersections != 0:
            mPCR6plus[Intersections - 1] += massConj
        else:
            Binary = tf.zeros((NbrSources, CardTheta), dtype=tf.int8)
            CardPC = tf.zeros((NbrSources, 1), dtype=tf.float32)

            for i in range(NbrSources):
                Binary[i, :] = tf.bitwise.bitwise_and(
                    tf.bitwise.right_shift(PC[i] + 1, tf.range(CardTheta - 1, -1, -1)),
                    1
                )
                CardPC[i, 0] = tf.reduce_sum(tf.cast(Binary[i, :] == 1, tf.float32))

            KeepIndex = tf.zeros((NbrSources, 1), dtype=tf.float32)
            for j in range(NbrSources):
                delta = []
                for js in range(NbrSources):
                    if CardPC[js] >= CardPC[j]:
                        for jp in range(NbrSources):
                            if PC[jp] != PC[js] and CardPC[jp] <= CardPC[js]:
                                if tf.reduce_sum(Binary[jp, :] <= Binary[js, :]) == CardTheta:
                                    delta.append(1)
                                else:
                                    delta.append(0)
                if not delta:
                    KeepIndex[j, 0] = 1
                else:
                    KeepIndex[j, 0] = 1 - tf.reduce_prod(tf.constant(delta))

            KeepIndex = tf.reshape(KeepIndex, [-1])
            IgnoringSetOfFE = tf.where(KeepIndex == 0)
            IgnoringSetOfFE = tf.reshape(IgnoringSetOfFE, [-1])
            masseConj = tf.gather(masseConj, tf.where(tf.logical_not(tf.math.equal(tf.range(tf.shape(masseConj)[0]), IgnoringSetOfFE))))
            PC = tf.gather(PC, tf.where(tf.logical_not(tf.math.equal(tf.range(tf.shape(PC)[0]), IgnoringSetOfFE))))

            for s in range(tf.shape(masseConj)[0]):
                Proportion = masseConj[s] * (massConj / tf.reduce_sum(masseConj, axis=0))
                index = PC[s] % CardTheta
                mPCR6plus[index] += Proportion

    return mPCR6plus

In [ ]:
# Call the PCR6fastfusion function
m1 = tf.constant([0.8, 0.1, 0.1], dtype=tf.float32)
m2 = tf.constant([0.7, 0.2, 0.1], dtype=tf.float32)
m3 = tf.constant([0.6, 0.3, 0.1], dtype=tf.float32)
m4 = tf.constant([0.2, 0.3, 0.5], dtype=tf.float32)
m5 = tf.constant([0.1, 0.5, 0.4], dtype=tf.float32)
m6 = tf.constant([0.3, 0.65, 0.05], dtype=tf.float32)

# Create BBA matrix
BBA = tf.transpose(tf.stack([m1, m2, m3, m4, m5, m6]))
print(BBA.shape)
print("BBA with", BBA.shape[1], "sources of information, each having", BBA.shape[0], "mass functions")

mPCR6 = PCR6plusfusion(BBA)
print("##############################")
print(mPCR6)
print("Sum of masses: ", tf.reduce_sum(mPCR6[:, 0]))
